In [1]:
! pip install torchinfo optuna

In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date, timedelta
import math
from torchinfo import summary
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [2]:
train_df = pd.read_parquet("train.parquet")

print(train_df.shape)
train_df.head()

(69687, 146)


,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,booking_changes,adr,required_car_parking_spaces,...,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,season_Spring,season_Summer,season_Winter
0,1,0.976909,-0.801038,2.242967,-0.299550,-0.10083,-0.176313,-0.413398,1.657600,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,1.630607,-0.700974,-0.469280
1,0,-0.001882,-0.801038,0.245144,1.939986,-0.10083,-0.176313,-0.413398,0.799488,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,1.426586,-0.469280
2,1,-0.001882,-0.801038,0.245144,4.179522,-0.10083,-0.176313,-0.413398,3.054525,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,-0.700974,-0.469280
3,1,0.976909,0.684562,0.245144,-0.299550,-0.10083,-0.176313,-0.413398,-0.324039,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,1.630607,-0.700974,-0.469280
4,0,-0.001882,-0.801038,0.245144,-0.299550,-0.10083,-0.176313,-0.413398,-1.267362,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,-0.700974,2.130922


In [3]:
eval_df = pd.read_parquet('valid.parquet')

print(eval_df.shape)
eval_df.head()

(10092, 146)


,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,booking_changes,adr,required_car_parking_spaces,...,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,season_Spring,season_Summer,season_Winter
0,0,0.976909,-1.296238,-1.752678,-0.29955,-0.10083,-0.176313,-0.413398,0.140938,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,1.426586,-0.469280
1,1,-0.001882,0.189362,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,0.260674,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,-0.700974,-0.469280
2,0,-0.001882,-1.296238,-1.752678,-0.29955,-0.10083,5.671727,-0.413398,-0.717173,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,1.630607,-0.700974,-0.469280
3,1,0.976909,-0.801038,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,-0.138447,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,1.630607,-0.700974,-0.469280
4,0,-0.001882,-1.296238,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,-1.355768,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.47126,-0.40217,-0.613269,-0.700974,2.130922


In [4]:
test_df = pd.read_parquet('test.parquet')

print(test_df.shape)
test_df.head()

(10171, 146)


,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,booking_changes,adr,required_car_parking_spaces,...,assigned_room_type_I,assigned_room_type_K,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,season_Spring,season_Summer,season_Winter
0,0,-0.001882,-0.801038,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,0.081070,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,-2.121972,2.486513,1.630607,-0.700974,-0.46928
1,0,-0.980673,-0.305838,-1.752678,-0.29955,-0.10083,-0.176313,4.925791,-1.515416,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.471260,-0.402170,1.630607,-0.700974,-0.46928
2,0,-0.001882,-0.801038,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,0.041158,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,0.471260,-0.402170,-0.613269,1.426586,-0.46928
3,0,-0.980673,0.189362,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,-0.757085,-0.296177,...,-0.062134,-0.045663,-0.118188,-0.036752,-0.077495,-2.121972,2.486513,-0.613269,-0.700974,-0.46928
4,1,-0.001882,-0.801038,0.245144,-0.29955,-0.10083,-0.176313,-0.413398,-0.876822,-0.296177,...,-0.062134,-0.045663,8.461124,-0.036752,-0.077495,-2.121972,2.486513,-0.613269,1.426586,-0.46928


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import optuna
from sklearn.metrics import log_loss, f1_score, roc_auc_score
import numpy as np

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_cols = [c for c in train_df.columns if c != 'is_canceled']
input_len = len(data_cols)

/opt/conda/envs/rapids/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class BookingDataset(Dataset):
    def __init__(self, df):
        X = df[data_cols].values.astype(np.float32)
        y = df['is_canceled'].values.astype(np.float32)
        self.X = torch.from_numpy(X).unsqueeze(1)  # shape (N,1,F)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx].to(device), self.y[idx].to(device)

# DataLoaders
batch_size = 512
train_ds = BookingDataset(train_df)
eval_ds = BookingDataset(eval_df)
dl_train = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
dl_val = DataLoader(eval_ds, batch_size=batch_size)

In [8]:
def build_model(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    filters = [trial.suggest_int(f'n_filters_l{i}', 16, 128, step=16) for i in range(n_layers)]
    kernel_size = trial.suggest_int('kernel_size', 2, 5)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)

    layers = []
    in_channels = 1
    length = input_len
    for nf in filters:
        layers.append(nn.Conv1d(in_channels, nf, kernel_size))
        layers.append(nn.ReLU())
        layers.append(nn.MaxPool1d(2))
        length = (length - (kernel_size - 1)) // 2
        in_channels = nf
    conv_net = nn.Sequential(*layers)

    flat_dim = in_channels * length
    fc_units = trial.suggest_int('fc_units', 32, 256, step=32)

    model = nn.Sequential(
        conv_net,
        nn.Flatten(),
        nn.Linear(flat_dim, fc_units),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_units, 1),
        nn.Sigmoid()
    ).to(device)

    return model

In [11]:
def objective(trial):
    model = build_model(trial)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    epochs = 10

    for epoch in range(epochs):
        model.train()
        for xb, yb in dl_train:
            optimizer.zero_grad()
            preds = model(xb).squeeze()
            loss = nn.functional.binary_cross_entropy(preds, yb)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, yb in dl_val:
            out = model(xb).squeeze()
            ys.append(yb.cpu().numpy())
            ps.append(out.cpu().numpy())
    y_true = np.concatenate(ys)
    y_prob = np.concatenate(ps)

    ll = log_loss(y_true, y_prob)
    f1 = f1_score(y_true, y_prob > 0.5)
    auc = roc_auc_score(y_true, y_prob)

    trial.set_user_attr('f1', f1)
    trial.set_user_attr('auc', auc)

    return ll

# Run Optuna study
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner()
)
study.optimize(objective, n_trials=50, n_jobs=2)

[I 2025-06-18 15:34:22,573] A new study created in memory with name: no-name-a78f8a39-2012-415b-8510-c782aacdbc24
[I 2025-06-18 15:39:36,873] Trial 1 finished with value: 0.3534345816380711 and parameters: {'n_layers': 2, 'n_filters_l0': 16, 'n_filters_l1': 128, 'kernel_size': 3, 'dropout': 0.34695545363843106, 'fc_units': 224, 'lr': 0.0005193789697629912}. Best is trial 1 with value: 0.3534345816380711.
[I 2025-06-18 15:39:36,951] Trial 0 finished with value: 0.3526614577332133 and parameters: {'n_layers': 3, 'n_filters_l0': 16, 'n_filters_l1': 96, 'n_filters_l2': 16, 'kernel_size': 4, 'dropout': 0.2675537928775241, 'fc_units': 192, 'lr': 0.003411355330444439}. Best is trial 0 with value: 0.3526614577332133.
[I 2025-06-18 15:40:33,827] Trial 3 finished with value: 0.35010755680364863 and parameters: {'n_layers': 2, 'n_filters_l0': 80, 'n_filters_l1': 64, 'kernel_size': 2, 'dropout': 0.11463030088467772, 'fc_units': 128, 'lr': 0.0013408666758295338}. Best is trial 3 with value: 0.35010

In [12]:
# Train final model on combined train+eval
full_df = pd.concat([train_df, eval_df]).reset_index(drop=True)
full_ds = BookingDataset(full_df)
dl_full = DataLoader(full_ds, batch_size=batch_size, shuffle=True)

best_params = study.best_trial.params
fixed_trial = optuna.trial.FixedTrial(best_params)
final_model = build_model(fixed_trial)
optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'])

for epoch in range(10):
    final_model.train()
    for xb, yb in dl_full:
        optimizer.zero_grad()
        loss = nn.functional.binary_cross_entropy(final_model(xb).squeeze(), yb)
        loss.backward()
        optimizer.step()

In [13]:
# Test evaluation
test_ds = BookingDataset(test_df)
dl_test = DataLoader(test_ds, batch_size=batch_size)

ys, ps = [], []
final_model.eval()
with torch.no_grad():
    for xb, yb in dl_test:
        out = final_model(xb).squeeze()
        ys.append(yb.cpu().numpy())
        ps.append(out.cpu().numpy())

y_true = np.concatenate(ys)
y_prob = np.concatenate(ps)

print('Test log_loss:', log_loss(y_true, y_prob))
print('Test F1     :', f1_score(y_true, y_prob > 0.5))
print('Test AUC    :', roc_auc_score(y_true, y_prob))

Test log_loss: 0.34260510358116797
Test F1     : 0.7227234042553191
Test AUC    : 0.9125014833247729


In [ ]:
# Save the trained model
save_path = 'neural_net_booking_model.pth'
torch.save(final_model.state_dict(), save_path)
print(f'Model saved to {save_path}')

Model saved to cnn_booking_model.pth
